In [2]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import cv2 as cv

import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.utils import shuffle


In [3]:
plant_names = [
    "Ulmus carpinifolia",
    "Acer",
    "Salix aurita",
    "Quercus",
    "Alnus incana",
    "Betula pubescens",
    "Salix alba 'Sericea'",
    "Populus tremula",
    "Ulmus glabra",
    "Sorbus aucuparia",
    "Salix sinerea",
    "Populus",
    "Tilia",
    "Sorbus intermedia",
    "Fagus silvatica"
]


In [4]:
def load_images() -> pd.DataFrame:
    data = []
    for directory in os.listdir('../data/'):
      label = int(directory[4:]) - 1
      for imagepath in os.listdir('../data/' + directory):
          image = Image.open(os.path.join('../data/', directory + '/'+ imagepath))
          new_image = np.array(cv.resize(np.asarray(image),(128, 128)))
          data.append((new_image, label))
          # data.append(((cv.resize(np.asarray(image),(64,64)).flatten() / 255).astype('float32'), label))
          image.close()
    return pd.DataFrame(data, columns=['images','labels'])

df = load_images()
df = shuffle(df)


In [29]:
def build_model():
    model = keras.Sequential([
        layers.Rescaling(1 / 255),
        layers.Conv2D(4, 3, input_shape=(128, 128, 3), activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(15, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model

In [6]:
X_train = tf.convert_to_tensor(df['images'].to_list())
y_train = tf.convert_to_tensor(df['labels'].to_list())

In [30]:
model = build_model()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3)

Epoch 1/10
25/25 [==============================] - 3s 70ms/step - loss: 1.6276 - accuracy: 0.5667 - val_loss: 8.8844 - val_accuracy: 0.1657
Epoch 2/10
25/25 [==============================] - 2s 66ms/step - loss: 0.9620 - accuracy: 0.8386 - val_loss: 3.5494 - val_accuracy: 0.1006
Epoch 3/10
25/25 [==============================] - 2s 70ms/step - loss: 0.5771 - accuracy: 0.9263 - val_loss: 3.6814 - val_accuracy: 0.2426
Epoch 4/10
25/25 [==============================] - 2s 67ms/step - loss: 0.2679 - accuracy: 0.9873 - val_loss: 3.7842 - val_accuracy: 0.1450
Epoch 5/10
25/25 [==============================] - 2s 69ms/step - loss: 0.1258 - accuracy: 1.0000 - val_loss: 3.8031 - val_accuracy: 0.1982
Epoch 6/10
25/25 [==============================] - 2s 79ms/step - loss: 0.0809 - accuracy: 1.0000 - val_loss: 2.6949 - val_accuracy: 0.3669
Epoch 7/10
25/25 [==============================] - 2s 81ms/step - loss: 0.0518 - accuracy: 1.0000 - val_loss: 3.2930 - val_accuracy: 0.2426
Epoch 8/10
25